In [9]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline


from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction import DictVectorizer

In [3]:
df = pd.read_csv("data.csv")
df.columns = df.columns.str.lower().str.replace(" ","_")

df["fbs"] = df["fbs"].map({True: 'true', False: 'false'})
df["exang"] = df["exang"].map({True: 'true', False: 'false'})

string = list(df.dtypes[df.dtypes == 'object'].index)

for c in string:
    df[c] = df[c].str.lower().str.replace(' ','_')
    
df["target"] = (df['num']>0).astype(int)

In [4]:
numerical = ['age',
 'trestbps',
 'chol',
 'thalch',
 'oldpeak',
 'ca'
 ]

categorical = [
 'sex',
 'cp',
 'fbs',
 'restecg',
 'exang',
 'slope',
 'thal',
]

In [5]:
del df['num']
del df['id']
del df['dataset']

In [6]:
for n in numerical:
    df[n] = df[n].fillna(df[n].median())

for n in categorical:
    df[n] = df[n].fillna(df[n].mode()[0])

In [7]:
df.isnull().sum()

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalch      0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64

In [8]:
df

,age,sex,cp,trestbps,chol,fbs,restecg,thalch,exang,oldpeak,slope,ca,thal,target
0,63,male,typical_angina,145.0,233.0,true,lv_hypertrophy,150.0,false,2.3,downsloping,0.0,fixed_defect,0
1,67,male,asymptomatic,160.0,286.0,false,lv_hypertrophy,108.0,true,1.5,flat,3.0,normal,1
2,67,male,asymptomatic,120.0,229.0,false,lv_hypertrophy,129.0,true,2.6,flat,2.0,reversable_defect,1
3,37,male,non-anginal,130.0,250.0,false,normal,187.0,false,3.5,downsloping,0.0,normal,0
4,41,female,atypical_angina,130.0,204.0,false,lv_hypertrophy,172.0,false,1.4,upsloping,0.0,normal,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
915,54,female,asymptomatic,127.0,333.0,true,st-t_abnormality,154.0,false,0.0,flat,0.0,normal,1
916,62,male,typical_angina,130.0,139.0,false,st-t_abnormality,140.0,false,0.5,flat,0.0,normal,0
917,55,male,asymptomatic,122.0,223.0,true,st-t_abnormality,100.0,false,0.0,flat,0.0,fixed_defect,1
918,58,male,asymptomatic,130.0,385.0,true,lv_hypertrophy,140.0,false,0.5,flat,0.0,normal,0


In [ ]:
df_full train, df_test = train_test_split